# DSC 10 Final Project - Music Classification. Due Saturday, June 9 at 11:59pm.
Welcome to the DSC 10 Final Project!  You will build a classifier that guesses whether a song is hip-hop or country, using only the numbers of times words appear in the song's lyrics.  By the end of the project, you should know how to:

1. Build a k-nearest-neighbors classifier.
2. Test a classifier on data.
3. Evaluate different sets of features.

Before you begin, execute the following cell to load the provided tests. Each time you start your server, you will need to execute this cell again to load the tests.

In [ ]:
# Run this cell, but please don't change it.

# These lines import the numpy and datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
import matplotlib.patches as mpatches
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# These lines load the tests.
from client.api.notebook import Notebook
project = Notebook('project.ok')
_ = project.auth(inline=True)

**Important:** The `ok` tests don't usually tell you that your answer is correct. More often, they help catch careless mistakes. It's up to you to ensure that your answer is correct. If you're not sure, ask someone (not for the answer, but for some guidance about your approach).

Once you're finished, you must do two things:
## a. Turn into OK

Select "Save and Checkpoint" in the File menu and then execute the submit cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission.

In [ ]:
_ = project.submit()

## b. Turn PDF into Gradescope

Select File > Download As > PDF via LaTeX in the File menu. Turn in this PDF file into the respective assignment at https://gradescope.com/.
If you submit more than once before the deadline, we will only grade your final submission.


# 0. The Dataset

Our dataset is a table of songs, each with a title, an artist, and a genre.  For each song, we also know how frequently certain words occur in that song.  More precisely, we have a list of approximately 5000 words that appear commonly in songs.  For each of these words, for each song, each item in the table describes the proportion of the song's lyrics that are the particular word.

For example, the lyrics of "In Your Eyes" is 168 words long. The word "like" appears twice:  $\frac{2}{168} \approx 0.0119$ of the words in the song. Similarly, the word "love" appears 10 times: $\frac{10}{168} \approx 0.0595$ of the words. Therefore,  
`lyrics.where("Title", "In Your Eyes").column("like").item(0)`  
is about $0.0119$, and  
`lyrics.where("Title", "In Your Eyes").column("love").item(0)`  
is about $0.0595$.

Our dataset doesn't contain all information about a song.  For example, it doesn't include the total number of words in each song, or information about the order of words in the song, let alone the melody, instruments, or rhythm. Nonetheless, you may find that word proportions alone are sufficient to build an accurate genre classifier.

Run the cell below to read the `lyrics` table. **It may take up to a minute to load.**

In [ ]:
# Just run this cell.
lyrics = Table.read_table('data/lyrics_clean.csv')

# The first 5 rows and 8 columns of the table
lyrics.select(np.arange(8)).show(5)

This dataset was extracted from the Million Song Dataset (http://labrosa.ee.columbia.edu/millionsong/) and combined with complementary datasets from musiXmatch (http://labrosa.ee.columbia.edu/millionsong/musixmatch) and Last.fm (http://labrosa.ee.columbia.edu/millionsong/lastfm).

The counts of common words in the lyrics for all of these songs are provided by the musiXmatch dataset (called a bag-of-words format). Only the top 5000 most common words are represented. For each song and for each word, we divided the number of occurrences of that word in the song by the total number of times that one of the 5000 most common words occured in the lyrics of that song.

The Last.fm dataset contains multiple tags for each song in the Million Song Dataset. Some of the tags are genre-related, such as "pop", "rock", "classic", etc.  To obtain our dataset, we first extracted songs with Last.fm tags that included the words "country", or "hip" and "hop". These songs were then cross-referenced with the musiXmatch dataset, and only songs with musixMatch lyrics were placed into our dataset. Finally, inappropriate words and songs with naughty titles were removed, leaving us with 4976 words in the vocabulary and 1721 songs.

In [ ]:
#The number of songs.
lyrics.num_rows

In [ ]:
#The number of words. The first three columns do not contain words.
lyrics.num_columns - 3

All titles are unique. The `row_for_title` function provides fast access to the row associated with a given title. Try it out to see how much data is stored in each row of this table!  

You may notice that we are using some new methods for getting a particular row, `index_by` and `get`.  The first line of code sets the "Title" column to be the index of the rows (normally, the index would be the range 0 to num_rows - 1).  Then, the body of the function `row_for_title` simply returns whatever is at the specified index.  This strategy tends to be slightly faster then using `where`, since `where` checks each row for the equality, whereas `get` does not need to perform many computations.  

Don't worry, you are not responsible for knowing these methods, that's why they are provided.

In [ ]:
# Look up the row corresponding to a given song title.
title_index = lyrics.index_by('Title')
def row_for_title(title):
    return title_index.get(title)[0]

# Find the row for the song "Right There".
right_there = row_for_title('Right There')
right_there

**Question 0.0.1:** Great!  Going one step further, what if we want to find the proportion of words in this song that are "brother"?  Without referencing the full `lyrics` table, write a line of code that finds this proportion using the row corresponding to the song "Right There".  Assign this proportion to the variable `proportion_brother` below.

*Hint:* The `Row` type is similar to an array, where the indices are the column labels.  You can read the [datascience documentation](http://data8.org/datascience/tutorial.html#accessing-values) to learn more about the `Row` object.

In [ ]:
proportion_brother = ...
proportion_brother

In [ ]:
_ = project.grade('q0_0_1')

**Question 0.0.2:** Set `expected_row_sum` to the number that you expect will result from summing all proportions in each row (this excludes the first three columns). 

In [ ]:
# Set row_sum to a number that's the (approximate) sum of each row of word proportions.
expected_row_sum = ...
expected_row_sum

In [ ]:
_ = project.grade('q0_0_2')

**Sanity check (optional & ungraded):** To check that the actual row sums are close to what you expect, draw the histogram below.

In [ ]:
# Run this cell to display a histogram of the sums of proportions in each row.
# This computation might take up to a minute; you can skip it if it's too slow.
Table().with_column('sums', lyrics.drop([0, 1, 2]).apply(sum)).hist(0)

## 0.1. Word Stemming
The columns other than 'Title', 'Artist', and 'Genre' in the `lyrics` table are all words that appear in some of the songs in our dataset.  Some of those names have been *stemmed*, or abbreviated heuristically, in an attempt to make different [inflected](https://en.wikipedia.org/wiki/Inflection) forms of the same base word into the same string.  For example, the column "manag" is the sum of proportions of the words "manage", "manager", "managed", and "managerial" (and perhaps others) in each song.  

Stemming makes it a little tricky to search for the words you want to use, so we have provided another table that will let you see examples of unstemmed (original) versions of each stemmed word.  Run the code below to load it.

In [ ]:
# Just run this cell to display a portion of vocab_table.
vocab_mapping = Table.read_table('data/mxm_reverse_mapping_safe.csv')
stemmed = np.take(lyrics.labels, np.arange(3, len(lyrics.labels)))
vocab_table = Table().with_column('Stem', stemmed).join('Stem', vocab_mapping)
vocab_table.take(range(900, 910))

**Question 0.1.1:** Assign `unchanged` to the **percentage** of words in the full table `vocab_table` that are the same as their stemmed form (such as "coup" above). 

In [ ]:
#Give a percentage between 0 and 100, not a proportion between 0 and 1.
unchanged = ...
unchanged

In [ ]:
_ = project.grade('q0_1_1')

**Question 0.1.2:** Assign `stemmed_message` to the stemmed version of the word "message".

In [ ]:
# Set stemmed_message to the stemmed version of "message" (which
# should be a string).  Use vocab_table.
stemmed_message = ...
stemmed_message

In [ ]:
_ = project.grade('q0_1_2')

**Question 0.1.3:** Assign `unstemmed_singl` to the word in `vocab_table` that has "singl" as its stemmed form. (*Note that multiple English words may stem to "singl", but only one example appears in `vocab_table`.*)

In [ ]:
# Set unstemmed_singl to the unstemmed version of "singl" (which
# should be a string).
unstemmed_singl = ...
unstemmed_singl

In [ ]:
_ = project.grade('q0_1_3')

**Question 0.1.4:** What word in `vocab_table` was shortened the most by this stemming process? Assign `most_shortened` to the word. The answer will provide an example of how heuristic stemming can collapse two unrelated words into the same stem (which is bad, but happens a lot in practice anyway).

In [ ]:
#Find the single word that got shortened more than any other word.
most_shortened = ...
vocab_table.where('Word', most_shortened)

In [ ]:
_ = project.grade('q0_1_4')

## 0.2. Splitting the dataset
We're going to use our `lyrics` dataset for three purposes.  First, we want to *train* several different song genre classifiers.  Second, we want to *validate* which classifier is most effective. Finally, we want to *test* the performance of our final classifier. Hence, we need three different datasets: *training*, *validation*, and *test*.

The purpose of a classifier is to learn from training data and generalize to unseen data that is similar to the training data. Therefore, we must ensure that no song appears in more than one of the three different sets. We do so by splitting the dataset randomly. The dataset has already been permuted randomly, so it's easy to split.  We just take the first part for training, the next part for validation, and the last for test. 

Run the code below (without changing it) to separate the three datasets into tables.

In [ ]:
# Here we have defined the proportion of our data
# that we want to designate for training as 11/16ths
# of our total dataset.  2/16ths of the data is
# reserved for validation.  The remaining 3/16ths
# will be used for testing.

training_proportion = 11/16
validation_proportion = 2/16
num_songs = lyrics.num_rows

num_train = int(num_songs * training_proportion)
num_valid = int(num_songs * validation_proportion)
num_test = num_songs - num_train - num_valid

train_lyrics = lyrics.take(range(num_train))
valid_lyrics = lyrics.take(range(num_train, num_train + num_valid))
test_lyrics = lyrics.take(range(num_train + num_valid, num_songs))

print("Training: ",   train_lyrics.num_rows, ";",
      "Validation: ", valid_lyrics.num_rows, ";",
      "Test: ",       test_lyrics.num_rows)

**Question 0.2.1.** Define a function `country_proportion` which takes as input a table of song lyrics and returns the proportion of country songs in that table.

In [ ]:
def country_proportion(table):
    """Return the proportion of songs in a table that have the Country genre."""
    return ...

In [ ]:
_ = project.grade('q0_2_1')

**Question 0.2.2.** Draw a horizontal bar chart with three bars that shows the proportion of country songs in each of the three datasets above.  Create a table called `country_table` and use this table to generate your bar chart.

In [ ]:
#Define a table from which to create your bar chart.
country_table = ...
country_table

# 1. K-Nearest Neighbors - a Guided Example

K-Nearest Neighbors (k-NN) is a classification algorithm.  A classification algorithm looks at many examples with assigned categories. Given an unseen example, it decides whether that example belongs to one or the other of two categories based on its similarity to previously seen examples in each category. This decision is based on some *features* of the example.  

A feature that we have about each song is *the proportion of times a particular word appears in the lyrics*, and the categories are two music genres: hip-hop and country.  The algorithm requires many previously seen examples for which both the features and categories are known: that's the `train_lyrics` table. 

We're going to visualize the algorithm, instead of just describing it. To get started, let's pick colors for the genres.

In [ ]:
# Just run this cell to define genre_color.

def genre_color(genre):
    """Assign a color to each genre."""
    if genre == 'Country':
        return 'gold'
    elif genre == 'Hip-hop':
        return 'blue'
    else:
        return 'green'

## 1.1. Classifying a  song

In k-NN, we classify a given song by finding the `k` songs in the *training set* that are most similar according to the features we choose. We call those songs with similar features the "neighbors".  The k-NN algorithm assigns the song to the most common category among its `k` neighbors.

Let's limit ourselves to just 2 features for now, so we can plot each song.  The features we will use are the proportions of the words "like" and "love" in the lyrics.  Taking the song "In Your Eyes" (in the test set), 0.0119 of its words are "like" and 0.0595 are "love". This song appears in the test set, so let's imagine that we don't yet know its genre.

First, we need to make our notion of similarity more precise.  We will say that the *dissimilarity*, or *distance* between two songs is the straight-line distance between them when we plot their features in a scatter diagram. This distance is called the Euclidean ("yoo-KLID-ee-un") distance.  

For example, in the song *Insane in the Brain* (in the training set), 0.0203 of all the words in the song are "like" and 0 are "love".  Its distance from *In Your Eyes* on this 2-word feature set is $\sqrt{(0.0119 - 0.0203)^2 + (0.0595 - 0)^2} \approx 0.06$.  (If we included more features or different features, the distance could be different.)

A third song, *Sangria Wine* (in the training set), is 0.0044 "like" and 0.0925 "love".

The function below creates a plot to display a test song and some training songs in a two-dimensional space defined by two features. As you can see in the result, *In Your Eyes* is more similar to *Sangria Wine* than to *Insane in the Brain*, as measured by distance in the plane.

In [ ]:
# Just run this cell.

def plot_with_two_features(test_song, training_songs, x_feature, y_feature):
    """Plot a test song and training songs using two features."""
    test_row = row_for_title(test_song)
    distances = Table().with_columns([
            x_feature, [test_row.item(x_feature)],
            y_feature, [test_row.item(y_feature)],
            'Color',   [genre_color('Unclassified')],
            'Title',   [test_song]
        ])
    for song in training_songs:
        row = row_for_title(song)
        color = genre_color(row.item('Genre'))
        distances.append([row.item(x_feature), row.item(y_feature), color, song])

    distances.scatter(x_feature, y_feature, labels='Title', s=200, color=distances.column('Color'))
    country_patch = mpatches.Patch(color=genre_color('Country'), label='Country')
    hiphop_patch = mpatches.Patch(color=genre_color('Hip-hop'), label='Hip-hop')
    unclass_patch = mpatches.Patch(color=genre_color('Unclassified'), label='Unclassified')
    plots.legend(handles=[country_patch, hiphop_patch, unclass_patch])
    
training = ["Sangria Wine", "Insane In The Brain"]
plot_with_two_features("In Your Eyes", training, "like", "love")

**Question 1.1.** Compute the Euclidean distance between the two country songs, *In Your Eyes* and *Sangria Wine* on the `['like', 'love']` feature set, and assign it to `country_distance`. 

In [ ]:
in_your_eyes = row_for_title("In Your Eyes")
sangria_wine = row_for_title("Sangria Wine")
country_distance = ...
country_distance

In [ ]:
_ = project.grade('q1_1')

The `plot_with_two_features` function can show the positions of several training songs. Below, we've added one that's even closer to *In Your Eyes*.

In [ ]:
training = ["Sangria Wine", "Lookin' for Love", "Insane In The Brain"]
plot_with_two_features("In Your Eyes", training, "like", "love")

**Question 1.2.** Complete the function `distance_two_features` that computes the Euclidean distance between any two songs, using two features. The last three lines call the `distance_two_features` function  to show that *Lookin' for Love* is closer to *In Your Eyes* than *Insane In The Brain*. 

In [ ]:
def distance_two_features(title0, title1, x_feature, y_feature):
    """Compute the distance between two songs, represented as rows."""
    row0 = ...
    row1 = ...
    return ...

for song in ["Lookin' for Love", "Insane In The Brain"]:
    song_distance = distance_two_features(song, "In Your Eyes", "like", "love")
    print(song, 'distance:\t', song_distance)

In [ ]:
_ = project.grade('q1_2')

The nearest neighbor to a song is the example in the training set that has the smallest distance from that song. In order to figure out the nearest neighbor, or the k nearest neighbors, we need to be able to compute all distances from our song to any other song. To do this, we'll use a new tool called a *higher-order function.*

A higher-order function is a function that either takes a function as an argument, or returns another function as its result.  Often times, higher-order functions are used so that the same method can be applied many times without needing to pass in all of the parameters each time, while still preserving the ability to change the parameters if need be.

The cell below has an example of a higher-order function. Make sure you understand this before moving on.

In [ ]:
# Define a function that takes the arguments you don't want to pass in every time
def add_number(original_number):
    # The body of this function defines an inner function 
    # that gives you the result you want, but only takes one argument
    def add_num(add_this):
        return original_number + add_this
    # Return the inner function
    return add_num

# Call the outer function with the parameters you only want to pass in once
# This defines a new function add_to_ten
add_to_ten = add_number(10)
# You can now call the function add_to_ten without passing in the argument 10 each time
print(add_to_ten(1), add_to_ten(2), add_to_ten(5))

**Question 1.3.** Define the higher-order function `distance_from` that takes as input a single song title and two features. It returns a function `for_song` that takes a second song title and computes the distance between the first and second songs.

This allows us to choose a song and two features, then compare it to any other song without needing to specify the original song and features again!  At some later point, if we want to change the original song or features, we are still able to.

*Hint:* Call `distance_two_features` in your solution rather than re-implementing its computation.

In [ ]:
def distance_from(title0, x_feature, y_feature):
    def for_song(title1):
        return ...
    return for_song

# Since distance_from returns a function, 
# the following line of code actually creates a 
# new function called distance_from_in_your_eyes.
distance_from_in_your_eyes = distance_from("In Your Eyes", "like", "love")

# Now we can calculate the distance from "In Your Eyes" 
# to any other song by simply inputting the name of the other song.
distance_from_in_your_eyes("Lookin' for Love")

In [ ]:
_ = project.grade('q1_3')

**Question 1.4.**  What are the names and genres of the 7 closest songs to "In Your Eyes" in  `train_lyrics`, where closeness is measured by Euclidean distance for the 2 features "like" and "love"?  To answer this question, make a table named `close_songs` containing a row for each those 7 songs, with columns "Title", "Artist", "Genre", "like", and "love" from the `train_lyrics` table, as well as a column called `distance` that contains the distance from "In Your Eyes" **sorted in ascending order**.

In [ ]:
like_love = train_lyrics.select(["Title", "Artist", "Genre", "like", "love"])
close_songs = ...
close_songs

In [ ]:
_ = project.grade('q1_4')

**Question 1.5 .** Write a function `most_common` that takes a `column_label` and a `table`. It returns the most common value in that column of that table. In case of a tie, it can return any of the most common values.

In [ ]:
def most_common(column_label, table):
    return ...

# Calling most_common classifies In Your Eyes as a country song, with 4 votes for Country to 3 for Hip Hop. 
# If not, you've made a mistake somewhere.
most_common('Genre', close_songs)

In [ ]:
_ = project.grade('q1_5')

Congratulations are in order -- you've classified your first song!

# 2. Features

Now, we're going to extend our classifier to consider more than two features at a time.

Euclidean distance still makes sense with more than two features. For *n* different features, we compute the difference between corresponding feature values for two songs, square each of the *n*  differences, sum up the resulting numbers, and take the square root of the sum.

**Question 2.0.1.** Write a function to compute the Euclidean distance between two *arrays* of features of *arbitrary* (but equal) length.  Use it to compute the distance between the first song in the training set and the first song in the test set, *using all of the features*.  (Remember that the title, artist, and genre of the songs are not features.) Store your answer in the variable `distance_first_to_first.`

*Hint:* You can convert a `Row` to an array using `np.array`, but keep in mind that all entries of an array must have the same type.

In [ ]:
def distance(features1, features2):
    """The Euclidian distance between two arrays of feature values."""
    ...

distance_first_to_first = ...
distance_first_to_first

In [ ]:
_ = project.grade('q2_0_1')

## 2.1. Creating your own feature set

Unfortunately, using all of the features has some downsides.  One clear downside is *computational* -- computing Euclidean distances takes a long time when we have lots of features.  So we're going to select just 20 for now.  We'd like to choose features that are very *discriminative*, that is, which lead us to correctly classify as much of the test set as possible.  This process of choosing features that will make a classifier work well is sometimes called *feature selection*, or more broadly *feature engineering*.

**Question 2.1.1.** Look through the list of features (the labels of the `lyrics` table after the first three).  Choose 20 that you think will let you distinguish pretty well between country and hip-hop songs.  You might want to come back to this question later to improve your list, once you've seen how to evaluate your classifier.  The first time you do this question, spend some time looking through the features, but not more than 15 minutes.

A good way to approach this is to look at the [song lyrics](https://www.azlyrics.com/lyrics/waylonjennings/justtosatisfyyou.html) for 'Just to Satisfy You' and some other songs, then chose words from those.  That way the features we choose are likely to show up in similar songs.

In [ ]:
lyrics.labels

In [ ]:
# Set my_20_features to a list of 20 features (strings that are column labels)
my_20_features = [ ]

train_20 = train_lyrics.select(my_20_features)
test_20 = test_lyrics.select(my_20_features)

In [ ]:
_ = project.grade('q2_1_1')

**Question 2.1.2** In a few sentences, describe how and why you selected your features. 

*I looked at words in 'Just to Satisfy You' and other country songs, plus thought about what songs might show up a lot in country songs.  I added those to my feature list, and also added some that I feel are pretty common in a lot of songs, like 'you', 'me', 'oh', and 'love' because these are nonzero, but the number of times they show up is probably different in the different genres.*

Let's make sure that you do not pick 20 rare words for your features. Run the following cell, and make sure at least three of your 20 words show up in more than one of the songs in the random sample. If not, go back and change the words in your feature set until you have at least three words that appear in more than one of the songs in the random sample.  Your classifier is likely to be more effective if you make an effort to use words that are not too rare.

In [ ]:
test_20.sample(10, with_replacement=False)

Now that we have selected our features, let's classify the first song from our test set using these features.  You can examine this song by running the cells below. Do you think it will be classified correctly?

In [ ]:
test_lyrics.take(0).select(['Title', 'Artist', 'Genre'])

In [ ]:
test_20.take(0)

As before, we want to look for the songs in the training set that are most like our test song.  We will calculate the Euclidean distances from the test song (using the 20 selected features) to all songs in the training set.  You could do this with `apply` or a `for` loop, but to make it computationally faster, we have provided a function, `fast_distances`, to do this for you.

You are not responsible for understanding the code in the cell below.

In [ ]:
# Just run this cell to define fast_distances.

# Takes 2 arguments: a table with a single row containing
# features of one test song (e.g., test_20.row(0)),
# and another table of features (for example, the whole
# table train_20).  Returns an array of distances,
# where the ith element is the distance from test_row to
# the ith row in counts_table.  
#
# This function is equivalent to using apply, but faster.
# It uses several techniques not covered in the course.
def fast_distances(test_row, train_rows):
    counts_matrix = np.asmatrix(train_rows.columns).transpose()
    diff = np.tile(np.array(test_row), [counts_matrix.shape[0], 1]) - counts_matrix
    distances = np.squeeze(np.asarray(np.sqrt(np.square(diff).sum(1))))
    return distances

**Question 2.1.3.** Use the `fast_distances` function provided above to compute the distance of the first song in the test set to all the songs in the training set.  Make a new table called `genre_and_distances` with one row for each song in the training set and two columns:
* The `"Genre"` of the training song
* The `"Distance"` from the first song in the test set 

Ensure that `genre_and_distances` is **sorted by increasing distance to the first test song**.

In [ ]:
distances = fast_distances(test_20.row(0), train_20)

genre_and_distances = ...
genre_and_distances

In [ ]:
_ = project.grade('q2_1_3')

**Question 2.1.4.**  Now compute the 5-nearest neighbors classification of the first song in the test set.  That is, decide on its genre by finding the most common genre among its 5 nearest neighbors, according to the distances you've calculated.  Then check whether your classifier chose the right genre.  (Depending on the features you chose, your classifier might not get this song right, and that's okay. You'll have a chance later to choose different features and maybe you can do a better job then.)

In [ ]:
# Set my_assigned_genre to the most common genre among these.
my_assigned_genre = ...

# Set my_assigned_genre_was_correct to True if my_assigned_genre
# matches the actual genre of the first song in the test set.
my_assigned_genre_was_correct = ...

print("The assigned genre: ", my_assigned_genre, "     Correct?", my_assigned_genre_was_correct )

In [ ]:
_ = project.grade('q2_1_4')

## 2.2. A classifier function

Now it's time to write a single function that encapsulates this whole process of classification.

**Question 2.2.1.** Write a function called `classify`.  It should take the following arguments:
* An array of features for a song to classify (e.g., `np.array(test_20.row(0))`),
* A table with a column for each feature and a row for each song in the training set (for example, `train_20`)
* An array of classes that has as many items as the previous table has rows, and in the same order, and
* `k`, the number of neighbors to use in classification.

It should return the class your classifier picks for the given row of features (e.g., `'Country'` or `'Hip-hop'`). 

In [ ]:
def classify(test_row, train_rows, train_classes, k):
    """Return the most common class among k nearest neighbors to test_row."""
    # Feel free to use these variable names or make up your own inside the function.
    distances = ...
    genre_and_distances = ...
    my_assigned_genre = ...
    return ...

In [ ]:
_ = project.grade('q2_2_1')

**Question 2.2.2.** Assign `grandpa_genre` to the genre predicted by your classifier for the song  "Grandpa Got Runned Over By A John Deere" in the test set, using 9 neighbors.

In [ ]:
grandpa_genre = ...
grandpa_genre

In [ ]:
_ = project.grade('q2_2_2')

**Question 2.2.3** To simplify things further, use the higher-order function `classify_using` that takes `train_rows`, `train_classes`, and `k` to define a function `simple_classify`. Assign `simple_classify` to the function that takes just one argument, a test row, and return the predicted genre of a song using `train_20` and five neighbors (`k=5`).  This way, when we classify a song, we just have to pass in the song's features, not the whole training set. 

Here is the example higher-order function that you saw earlier in this project. Reviewing this example should help you answer this question.

In [ ]:
# Define a function that takes the arguments you don't want to pass in every time
def add_number(original_number):
    # Define an inner function that gives you the result you want, but only takes one argument
    def add_num(add_this):
        return original_number + add_this
    # Return the inner function
    return add_num

# Call the outer function with the parameters you only want to pass in once
# This defines a new function add_to_ten
add_to_ten = add_number(10)
# You can now call the function add_to_ten without passing in the argument 10 each time
print(add_to_ten(1), add_to_ten(2), add_to_ten(5))

In [ ]:
def classify_using(train_rows, train_classes, k):
    def for_row(test_row):
        return classify(test_row, train_rows, train_classes, k)
    return for_row

simple_classify = ... # This is the only line you need to change.
simple_classify(np.array(test_20.row(0)))

In [ ]:
_ = project.grade('q2_2_3')

## 2.3. Evaluating your classifier

Now that it's easy to use the classifier, let's see how accurate it is on the whole test set.

**Question 2.3.1.** Classify every song in the test set (provided), then compute the proportion of correct classifications. 

In [ ]:
test_guesses = test_20.apply(simple_classify)
proportion_correct = ...
proportion_correct

In [ ]:
_ = project.grade('q2_3_1')

At this point, you've gone through one cycle of classifier design.  Let's summarize the steps:
1. From available data, select test and training sets.
2. Choose an algorithm you're going to use for classification.
3. Identify some features.
4. Define a classifier function using your features and the training set.
5. Evaluate its performance (the proportion of correct classifications) on the test set.

In practice, if the performance is good enough for your application, you might be done. In this case, we might be able to do better by focusing on step 3.

# 3. Feature design

One way to interpret the accuracy of a classifier is to compare it to another classifier. Here, we will consider other classifiers that use different features.

Above, you selected 20 features by hand that you thought would help distinguish country songs from hip-hop songs. Let's instead explore a data-driven approach to finding words that are highly indicative of one particular genre. We'll focus on finding words that are much more prevalent in country music, though you could do the same analysis for hip-hop.

We will calculate, for each word, the average proportion of that word among all songs of each genre. First, we separate the songs in the training set by genre and keep only the columns corresponding to words.

In [ ]:
country_words = train_lyrics.where('Genre', 'Country').drop(0, 1, 2)
hip_hop_words = train_lyrics.where('Genre', 'Hip-hop').drop(0, 1, 2)

**Question 3.0.1.** Complete the function `word_average` that takes as input a table `t`, either `country_words` or `hip_hop_words`, where each row of the table represents a song and each column of the table represents a word. The function should return an array with one entry for each word (column) in the table `t`, which comes from averaging the proportion of that word among all songs in the table.

In [ ]:
def word_average(t):
    return ...

In [ ]:
_ = project.grade('q3_0_1')

**Question 3.0.2.** Use the function `word_average` to calculate the average proportion of each word in each genre. Create a table called `words` with three columns:
* "Word" : a row for each word
* "Country Avg" : the average proportion of the word in country songs
* "Hip Hop Avg" : the average proportion of the word in hip-hop songs

In [ ]:
words = ...
words

In [ ]:
_ = project.grade('q3_0_2')

**Question 3.0.3.** Among words that are present in at least one country song and at least one hip-hop song, find the 10 words that have the highest ratio of "Country Avg" to "Hip Hop Avg." These 10 words should be highly indicative of a song being a country song. Store your 10 words in an array called `most_country`.

In [ ]:
most_country = ...
most_country

In [ ]:
_ = project.grade('q3_0_3')

**Question 3.0.4.** Using the 10 words in `most_country` as features, build a 5-nearest-neighbor classifier using these features and compute its accuracy on the test set. (You can write any code you want, as long as `proportion_correct_most_country` is computed correctly; the other names are only suggestions.)

In [ ]:
train_most_country = ...
test_most_country = ...
classify_most_country = ...
guesses_most_country =  ...
proportion_correct_most_country = ...
proportion_correct_most_country

In [ ]:
_ project.grade('q3_0_4')

**Question 3.0.5.** Is there anything random about a classifier's accuracy measured in this way?  Is it possible that a classifier's performance is due to chance?  If so, describe (in 2-3 sentences) how you would investigate that.

*Write your answer here, replacing this text.*

**Question 3.0.6.** Below we've provided 10 features selected by the staff.  Build a 5-nearest-neighbor classifier using these features and compute its accuracy on the test set. (You can write any code you want, as long as `proportion_correct_staff` is computed correctly; the other names are only suggestions.)

In [ ]:
features_staff = ["come", "do", "have", "heart", "make", "never", "now", "wanna", "with", "yo"]
train_staff = ...
test_staff = ...
classify_staff = ...
guesses_staff = ...
proportion_correct_staff = ...
proportion_correct_staff

In [ ]:
_ = project.grade('q3_0_6')

**Question 3.0.7.** Which classifier did better, the classifier using words that were highly indicative of country music, or the classifier of words selected by the staff? Is this surprising? Can you explain why this happened? What does this comparison tell you about selecting a good set of features?

*Write your answer here, replacing this text.*

## 3.1. Feature selection

What happens if we use only use a subset of the staff features? Is there a single feature that's better than the rest? Can removing features improve our classifier, or does removing features always hurt? Can removing different features degrade accuracy by different amounts?

As soon as you begin comparing the performance of many different classifiers, searching for the best one, it is important to conduct that search on the *validation set* rather than the *test set*.  

Conventionally, the test set is used as a final evaluation of the classifier. The validation set is used to adjust the classifier prior to the final evaluation, such as choosing a new set of features or a different value for `k`.  The validation set is called `valid_lyrics`.

Below we have defined a higher-order function `accuracy_on` that takes an evaluation set, either `valid_lyrics` or `test_lyrics`. It returns a function that takes a feature list and a value for `k`, and returns the accuracy (proportion correct) of a `k`-NN classifier on the evaluation set using those features.

In [ ]:
def accuracy_on(evaluation_set):
    def accuracy(features, k):
        train = train_lyrics.select(features)
        ev = evaluation_set.select(features)
        classifier = classify_using(train, train_lyrics.column('Genre'), k)
        guesses = ev.apply(classifier)
        return np.count_nonzero(guesses==evaluation_set.column('Genre'))/guesses.size
    return accuracy

valid_accuracy = accuracy_on(valid_lyrics)
valid_accuracy(features_staff, 5)

**Question 3.1.1.** Create a two-column table `solo_accuracies` that shows the accuracy on the validation set using 5 neighbors and only a single feature. Add a row for every feature in `staff_features`.

In [ ]:
solo_accuracies = Table(['Feature', 'Solo Accuracy'])
for f in features_staff:
    ...
solo_accuracies.sort(1, descending=True)

In [ ]:
_ = project.grade('q3_1_1')

An *ablation* study involves attempting to determine which features matter most for classification accuracy by removing ("ablating") each of them individually.

**Question 3.1.2.** Create a two-column table `ablation_accuracies` that shows the accuracy on the validation set of a 5-NN classifier that has all `staff_features` except one. Add a row for every feature in `staff_features` that you leave out. 

In [ ]:
ablation_accuracies = Table(['Feature', 'All-Except-One Accuracy'])
for f in features_staff:
    features = list(features_staff) # Make a copy of the staff features
    features.remove(f)              # Remove one feature from the copy
    ...
ablation_accuracies.sort(1, descending=True)

In [ ]:
_ = project.grade('q3_1_2')

**Question 3.1.3.** You should have found that both "make" and "yo" have a high solo accuracy, but they have very different effects on the classifier when removed. Which feature appears to be more important to the final classifier?

*Write your answer here, replacing this text.*

**Question 3.1.4.** Draw a scatter diagram with one dot for every **odd** value of `k` from 1 to 15 (inclusive) that plots the accuracy on the validation set on the horizontal axis and accuracy on the test set on the vertical axis. Use `features_staff` as the features for all points.

In [ ]:
k_accuracies = Table(['k', 'Validation Accuracy', 'Test Accuracy'])
test_accuracy = accuracy_on(test_lyrics)
...
...
k_accuracies.scatter(1, 2, labels=0, s=200)

**Question 3.1.5.** Does validation accuracy appear to be useful in selecting `k`, if what you care about most is classification accuracy on other unseen data (such as the test set)? Does the choice of `k` appear to matter at all? Explain your answer in 1 or 2 sentences.

*Write your answer here, replacing this text.*

## 3.2. Open Ended

Now that you know how to evaluate different feature sets, it's time to build the best classifier you can.

**Question 3.2.1.** Find a set of features that gives at least 85% accuracy on the validation set, or as close to that as you can get.  For now, `k` is set to `5`, but you can also change `k` to be some other odd number.

In [ ]:
better_features = ...
k = 5

print('Validation:', valid_accuracy(better_features, k), 'Test:', test_accuracy(better_features, k))

**Question 3.2.2.** Why do you think we have been using odd `k`'s throughout this project?  What would happen if used an even value of `k`? Explain your answer.

*Write your answer here, replacing this text.*

**Question 3.2.3.** Suppose the following table  of six songs were our entire dataset.  Explain what would happen to the results of our classifier if we tried classifying some song with `k = 5`.  In general, what is the issue with setting `k` to be a value very close to the size of the dataset?

In [ ]:
lyrics.take(range(2, 8)).show()

*Write your answer here, replacing this text.*

Congratulations! You're done with the required portion of the project.

## Don't forget to submit to both OK and Gradescope!

In [ ]:
_ = project.submit()

**Ungraded.** Feel free to create an even better classifier. You're not restricted to using only word proportions as features. For example, given the data, you could compute various notions of vocabulary size or estimated song length. If you think you built a classifier that works well, post on Piazza and let us know.

In [ ]:
#####################
# Custom Classifier #
#####################
